<br>

# Introdução

Converter de zip para tar

In [ ]:
#!pip install pyarrow        # Necessário para usar o parquet

In [ ]:
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [ ]:
from paths import *

<br>

## Converter csv to parquet

Os arquivos do dados abertos são gigantes e não dá para trabalhar com eles, nos formatos disponibilizados.
Logo, é necessário converter para .parquet ou outro formato comprimido.


Abaixo tem funções que poderão ser úteis no *dtypes*.
```python
types_dict = {'A': int, 'B': float}
types_dict.update({col: str for col in col_names if col not in types_dict})

pd.read_csv('file.csv', dtype=types_dict)
```

<br>

Inicialmente lemos os arquivos, definimos pasta de saída e parâmetros.

A função a seguir lê o arquivo zipado, que tem um arquivo *.csv* dentro; pega as colunas e ajusta elas, indicando que o dtype de todas será texto (nesse primeiro momento!)

Após isso define o tamanho dos *chunks*! e vai lendo chunk por chunk, inserindo em um arquivo *.parquet*. Ao final, salva esse aquivo!

In [ ]:
def convert_csv2parquet(input_file, output_path, encoding, sep):
    try:
        # Get File
        my_zipfile = os.path.basename(input_file)

        # Columns Names from csv file
        cols = pd.read_csv(
            os.path.join(input_file),
            sep=sep,
            encoding=encoding,
            low_memory=False,
            nrows=10,
            dtype=str, #TODO: Improve dtypes
        ).columns

        # Set schema from csv file: set all strings
        fields = []
        for col in list(cols):
            col_type = pa.field(col, pa.string()),
            fields.append(col_type[0])
        my_schema = pa.schema(fields)

        # Enumerate chunks to process
        df_enum = enumerate(
            pd.read_csv(
                os.path.join(input_file),
                sep=sep,
                encoding=encoding,
                low_memory=False,
                chunksize=10000,
                dtype=str,
            )
        )

        # Create Output Directory
        os.makedirs(output_path, exist_ok=True)

        # Write parquet in chunks
        pqwriter = None
        for i, df in enumerate(df_enum):
            table = pa.Table.from_pandas(
                df[-1],
                schema=my_schema,
            )

            # For the first chunk of records
            if i == 0:
                # Create a parquet write object giving it an output file
                pqwriter = pq.ParquetWriter(
                    os.path.join(output_path, '{}.parquet.gzip'.format(my_zipfile.split('.')[0])),
                    compression='gzip',
                    schema=my_schema,
                )
            pqwriter.write_table(table)

        # Close the parquet writer
        pqwriter.close()
        print('"{}" converter succeed!'.format(my_zipfile))
    
    except Exception as e:
        print(e)

In [ ]:
# Parameters
#input_file = os.path.join(controle_path, 'controle_mensal_parametros_basicos_2020.zip')
#output_path = os.path.join(input_path_parquet, 'controle')
#encoding = 'ISO-8859-1'
#sep = ';'

In [ ]:
#convert_csv2parquet(input_file, output_path, encoding, sep)
#df = pd.read_parquet(os.path.join(output_path, 'controle_mensal_parametros_basicos_2020.parquet.gzip'))
#df.head()

<br>

## Convert *csv* to *parquet*

Converte os dados obtidos em formato *csv* (inseridos dentro de um arquivo *zip*) para o formato *parquet*.<br>
Nessa primeira transformação não me preocupei com formatos, dtypes, renames etc.

In [ ]:
# Loop
paths = ['cadastro', 'controle', 'vigilancia']
for path in paths:
    # Paths
    path_in = os.path.join(bruto_path, path)
    path_out = os.path.join(input_path_parquet, path)
    
    # Loop
    list_files = os.listdir(path_in)
    for file in list_files:
        print('\n{}'.format(file))
        convert_csv2parquet(
            os.path.join(path_in, file),
            path_out,
            encoding='ISO-8859-1',
            sep=';',
        )

<br>

## Repartition

Com o formato *parquet*, reparticionei o arquivo para que o acesso fosse facilitado.

In [ ]:
# Parameters
paths = ['cadastro', 'controle', 'vigilancia']
#paths = [path for path in paths if path.startswith('con')]
paths

In [ ]:
for path in paths:
    # Parameters
    path_in = os.path.join(input_path_parquet, path)
    path_out = os.path.join(input_path_parquet_partitioned, path)
    os.makedirs(path_out, exist_ok=True)
    
    # Loop
    list_files = os.listdir(path_in)
    print(list_files)
    
    for file in list_files:
        file_out = os.path.basename(file).split('.')[0]
        print(file_out)
        df = pq.read_table(os.path.join(path_in, file))
        
        # Rename Columns
        cols = df.column_names
        df = df.rename_columns([x.strip().title() for x in cols])
        
        # Save Parquet Partitioned        
        pq.write_to_dataset(
            df,
            root_path=os.path.join(path_out, file_out),
            partition_cols=['Uf', 'Código Ibge']
        )